# Clean data about individual police officers

* Input: `raw_list_of_texas_officers.csv`
* Output: `list_of_texas_officers.csv`
  
##### Author: Everett Wetchler (everett.wetchler@gmail.com)

## 1. Setup and read data

In [1]:
DTW_PROJECT_KEY = 'tji/auxiliary-datasets'
RAW_DATAFRAME_NAME = 'raw_list_of_texas_officers'
AGENCY_COUNTY_DATAFRAME = 'agencies_and_counties'
CLEANED_FILENAME = 'list_of_texas_officers.csv'

In [2]:
import csv
import datadotworld as dw
import datetime
import numpy as np
import pandas as pd

%load_ext watermark
%watermark -a "Everett Wetchler" -d -t -z -r -g -w -p datadotworld,numpy,pandas

Everett Wetchler 2018-05-09 15:15:36 CDT

datadotworld 1.6.0
numpy 1.14.3
pandas 0.22.0
Git hash: 05f1bdd24121e53c09598c2b13f907393e03805a
Git repo: git@github.com:texas-justice-initiative/data-processing.git
watermark 1.6.0


In [3]:
from lib.standardize_police_agency_names import standardize_agency_name

In [4]:
datasets = dw.load_dataset(DTW_PROJECT_KEY, force_update=True)
agency_county = datasets.dataframes[AGENCY_COUNTY_DATAFRAME]
agency_county.head()

,agency,county
0,ANDERSON CO CONST PCT 1,ANDERSON
1,ANDERSON CO CONST PCT 2,ANDERSON
2,ANDERSON CO CONST PCT 3,ANDERSON
3,ANDERSON CO CONST PCT 4,ANDERSON
4,ANDERSON CO DIST ATTY OFFICE,ANDERSON


In [5]:
officer_info = datasets.dataframes[RAW_DATAFRAME_NAME]
officer_info.head()

,first_name,middle,last_name,suffix,current_department,highest_cert,service_time
0,BRADFORD,E,TIPPIT,NaN,MISSOURI CITY POLICE DEPT.,Master Peace Officer,23.92
1,ROGER,W,KENDALL,NaN,TEXAS BOARD OF CRIMINAL JUSTICE,Master Peace Officer,25.67
2,CHARLES,T,KELLEY,NaN,WILLIAMSON CO. SHERIFF'S OFFICE,Master Peace Officer,23.41
3,NAYA,C,POPE,NaN,IRVING POLICE DEPT.,Master Peace Officer,36.91
4,LINDA,F,HARTT-GOGGIN,NaN,ANTHONY POLICE DEPT.,Master Peace Officer,32.41


## 2. Begin cleaning

### Standardize agency names

In [6]:
officer_info['current_department'] = officer_info['current_department'].apply(standardize_agency_name)
officer_info.head()

,first_name,middle,last_name,suffix,current_department,highest_cert,service_time
0,BRADFORD,E,TIPPIT,NaN,MISSOURI CITY POLICE DEPT,Master Peace Officer,23.92
1,ROGER,W,KENDALL,NaN,TEXAS BOARD OF CRIMINAL JUSTICE,Master Peace Officer,25.67
2,CHARLES,T,KELLEY,NaN,WILLIAMSON CO SHERIFFS OFFICE,Master Peace Officer,23.41
3,NAYA,C,POPE,NaN,IRVING POLICE DEPT,Master Peace Officer,36.91
4,LINDA,F,HARTT-GOGGIN,NaN,ANTHONY POLICE DEPT,Master Peace Officer,32.41


### Add county names

In [7]:
agency_to_county = dict(zip(agency_county.agency, agency_county.county))
officer_info['current_department_county'] = officer_info['current_department'].apply(lambda d: agency_to_county.get(d, None))
officer_info['current_department_county'].isnull().value_counts()

False    77759
True        41
Name: current_department_county, dtype: int64

In [8]:
officer_info['current_department'][officer_info['current_department_county'].isnull()].value_counts()

ALABAMA-COUSHATTA TRIBE OF TEXAS POLICE DEPT    17
STAFFORD FIRE MARSHALS OFFICE                    6
HIDALGO CO CONST PCT 5                           5
TENAHA POLICE DEPT                               4
UNIV OF DALLAS POLICE DEPT                       2
RICHLAND POLICE DEPT                             2
COKE CO CONST PCT                                1
50TH JUDICIAL DIST ATTY OFFICE                   1
DUMAS FIRE DEPT                                  1
HAMILTON CO CONST PCT 3                          1
FRANKLIN CO FIRE MARSHALS OFFICE                 1
Name: current_department, dtype: int64

### Uppercase string values

In [9]:
for c in officer_info.columns:
    officer_info[c] = officer_info[c].apply(lambda val: val if not isinstance(val, str) else val.upper())
officer_info.head()

,first_name,middle,last_name,suffix,current_department,highest_cert,service_time,current_department_county
0,BRADFORD,E,TIPPIT,NaN,MISSOURI CITY POLICE DEPT,MASTER PEACE OFFICER,23.92,FORT BEND
1,ROGER,W,KENDALL,NaN,TEXAS BOARD OF CRIMINAL JUSTICE,MASTER PEACE OFFICER,25.67,WALKER
2,CHARLES,T,KELLEY,NaN,WILLIAMSON CO SHERIFFS OFFICE,MASTER PEACE OFFICER,23.41,WILLIAMSON
3,NAYA,C,POPE,NaN,IRVING POLICE DEPT,MASTER PEACE OFFICER,36.91,DALLAS
4,LINDA,F,HARTT-GOGGIN,NaN,ANTHONY POLICE DEPT,MASTER PEACE OFFICER,32.41,EL PASO


### Add 'full_name' column

In [10]:
officer_info['full_name'] = ''
for col in ['first_name', 'middle', 'last_name', 'suffix']:
    officer_info['full_name'] = officer_info['full_name'] + ' ' + officer_info[col].fillna('')
officer_info['full_name'] = officer_info['full_name'].apply(lambda s: ' '.join(s.strip().split()))
officer_info.sample(5)

,first_name,middle,last_name,suffix,current_department,highest_cert,service_time,current_department_county,full_name
56187,TROY,A,SANDIFER,NaN,WACO POLICE DEPT,ADVANCED PEACE OFFICER,7.92,MCLENNAN,TROY A SANDIFER
44012,ADRIAN,O,MARQUEZ,NaN,PECOS POLICE DEPT,ADVANCED PEACE OFFICER,11.75,REEVES,ADRIAN O MARQUEZ
13559,CHRISTOPHER,R,HERNDON,NaN,AUSTIN COMM COL DIST POLICE,MASTER PEACE OFFICER,23.41,TRAVIS,CHRISTOPHER R HERNDON
18943,ANTHONY,J,ELIAS,NaN,SAN ANTONIO POLICE DEPT,MASTER PEACE OFFICER,28.08,BEXAR,ANTHONY J ELIAS
35433,RICHARD,NaN,MORGAN,NaN,SAN ANTONIO POLICE DEPT,MASTER PEACE OFFICER,16.58,BEXAR,RICHARD MORGAN


### Notice there is no column for sex/gender. We will predict this from social security data
* Officers generally start in their early 20s, and rarely after 35 ([source](https://www.quora.com/What-is-the-average-age-for-people-joining-the-police-force))
* Since SSA data for names/genders is by birth year, we will assume, for simplicity, that all officers began their service at age 25. We will use their years of service to compute their likely birth year, and examine baby names from that year and nearby years to see how probable a given gender is for their first name.

#### Create a table to look up the probability of being male from the birth year

In [11]:
def get_p_male(dataset, y):
    raw = dataset.raw_data['original/yob%d.txt' % y]
    reader = csv.reader(raw.decode("utf-8").splitlines())
    df = pd.DataFrame(list(reader), columns=['name','sex','count'])
    df['name'] = df['name'].str.upper()
    df['count'] = df['count'].astype(int)
    df = df.pivot('name', 'sex', 'count').fillna(0).astype(int)
    df = df.div(df.sum(axis=1), axis=0)
    return df['M']

In [12]:
dataset = dw.load_dataset('us-ssa-gov/0b20b056-fda4-4926-9c78-e2fbddf0432f')

In [13]:
years = []
index = []
for y in range(1900, 2016):
    print(y, end=' ')
    years.append(get_p_male(dataset, y))
    index.append(y)

p_male_lookup = pd.DataFrame.from_records(years, index=index)
print(p_male_lookup.shape)

1900 1901 1902 1903 1904 1905 1906 1907 1908 1909 1910 1911 1912 1913 1914 1915 1916 1917 1918 1919 1920 1921 1922 1923 1924 1925 1926 1927 1928 1929 1930 1931 1932 1933 1934 1935 1936 1937 1938 1939 1940 1941 1942 1943 1944 1945 1946 1947 1948 1949 1950 1951 1952 1953 1954 1955 1956 1957 1958 1959 1960 1961 1962 1963 1964 1965 1966 1967 1968 1969 1970 1971 1972 1973 1974 1975 1976 1977 1978 1979 1980 1981 1982 1983 1984 1985 1986 1987 1988 1989 1990 1991 1992 1993 1994 1995 1996 1997 1998 1999 2000 2001 2002 2003 2004 2005 2006 2007 2008 2009 2010 2011 2012 2013 2014 2015 (116, 95013)


In [14]:
p_male_lookup.loc[[1950, 1990], ['MARY','JOHN','JOHNNIE','TAYLOR']].T

,1950,1990
MARY,0.001814,0.005166
JOHN,0.997751,0.997221
JOHNNIE,0.655528,0.860892
TAYLOR,1.000000,0.475309


In [15]:
known_names = set(p_male_lookup.columns)
p_male_overall = p_male_lookup.mean(axis=0)

#### Apply this to our data

In [16]:
data_collection_date = datetime.datetime(2018, 5, 1)
guess_birth_years = officer_info.service_time.apply(
    lambda years: (data_collection_date - datetime.timedelta(days=365.25*(years+25))).year)

In [17]:
ps = []
ypm = 2  # years plus or minus
for name, y in zip(officer_info.first_name, guess_birth_years):
    if pd.isnull(name) or name not in known_names:
        ps.append(np.nan)
    elif pd.isnull(y):
        ps.append(p_male_overall.loc[name])
    else:
        try:
            probs = p_male_lookup.loc[(y - ypm):(y + ypm), name]
            probs = [p for p in probs if pd.notnull(p)]
            if len(probs) <= ypm:
                # We don't have any recorded births for this name
                # in at least half of the years we checked. Let's
                # just use the all-time average for this name.
                ps.append(p_male_overall.loc[name])
            else:
                ps.append(np.mean(probs))
        except KeyError as e:
            ps.append(p_male_overall.loc[name])
officer_info['p_male'] = ps
print('No gender prediction for %d/%d (%.1f%%) officers with names not in SSA data' % (
    officer_info['p_male'].isnull().sum(), len(officer_info), officer_info['p_male'].isnull().mean() * 100))
officer_info.head()

No gender prediction for 1337/77800 (1.7%) officers with names not in SSA data


,first_name,middle,last_name,suffix,current_department,highest_cert,service_time,current_department_county,full_name,p_male
0,BRADFORD,E,TIPPIT,NaN,MISSOURI CITY POLICE DEPT,MASTER PEACE OFFICER,23.92,FORT BEND,BRADFORD E TIPPIT,0.996006
1,ROGER,W,KENDALL,NaN,TEXAS BOARD OF CRIMINAL JUSTICE,MASTER PEACE OFFICER,25.67,WALKER,ROGER W KENDALL,0.994526
2,CHARLES,T,KELLEY,NaN,WILLIAMSON CO SHERIFFS OFFICE,MASTER PEACE OFFICER,23.41,WILLIAMSON,CHARLES T KELLEY,0.993646
3,NAYA,C,POPE,NaN,IRVING POLICE DEPT,MASTER PEACE OFFICER,36.91,DALLAS,NAYA C POPE,0.000000
4,LINDA,F,HARTT-GOGGIN,NaN,ANTHONY POLICE DEPT,MASTER PEACE OFFICER,32.41,EL PASO,LINDA F HARTT-GOGGIN,0.002594


### Add an integer column for certification level

In [18]:
officer_info['highest_cert'] = officer_info['highest_cert'].fillna('NONE')
cert_levels = {
    'NONE': 0,
    'BASIC PEACE OFFICER': 1,
    'INTERMEDIATE PEACE OFFICER': 2,
    'ADVANCED PEACE OFFICER': 3,
    'MASTER PEACE OFFICER': 4,
}
officer_info['highest_cert_int'] = officer_info['highest_cert'].apply(lambda c: cert_levels[c])

### Reorder and rename columns, sort values

In [19]:
officer_info = officer_info[[
    'first_name', 'middle', 'last_name', 'suffix', 'full_name',
    'current_department', 'current_department_county',
    'highest_cert', 'highest_cert_int', 'service_time', 'p_male']]

officer_info.columns = [
    'name_first', 'name_middle', 'name_last', 'name_suffix', 'name_full',
    'current_agency', 'current_agency_county',
    'highest_cert', 'highest_cert_int', 'service_time', 'p_male']

officer_info.sort_values(['current_agency_county', 'current_agency', 'name_full'], inplace=True)
officer_info.sample(10)

,name_first,name_middle,name_last,name_suffix,name_full,current_agency,current_agency_county,highest_cert,highest_cert_int,service_time,p_male
2531,JOSEPH,B,NGUYEN,NaN,JOSEPH B NGUYEN,HOUSTON POLICE DEPT,HARRIS,MASTER PEACE OFFICER,4,24.58,0.995080
72207,ANTONIO,NaN,MARTINEZ,NaN,ANTONIO MARTINEZ,INDIAN LAKE POLICE DEPT,CAMERON,NONE,0,0.17,0.993318
49116,VICTOR,NaN,HERNANDEZ,NaN,VICTOR HERNANDEZ,FORT WORTH POLICE DEPT,TARRANT,ADVANCED PEACE OFFICER,3,10.00,0.990265
38091,RHONDA,L,WILSON,NaN,RHONDA L WILSON,HOUSTON POLICE DEPT,HARRIS,MASTER PEACE OFFICER,4,15.42,0.004637
35856,MARTIN,G,MONTALVO,NaN,MARTIN G MONTALVO,HOUSTON POLICE DEPT,HARRIS,MASTER PEACE OFFICER,4,16.42,0.991717
23194,JEFFREY,G,MIDDLETON,NaN,JEFFREY G MIDDLETON,PEARLAND POLICE DEPT,BRAZORIA,MASTER PEACE OFFICER,4,21.33,0.995609
38180,JOSE,J,ROCHA,NaN,JOSE J ROCHA,STARR CO SHERIFFS OFFICE,STARR,MASTER PEACE OFFICER,4,14.08,0.989227
49677,QUENTIN,M,WILLIAMS,NaN,QUENTIN M WILLIAMS,TOM GREEN CO SHERIFFS OFFICE,TOM GREEN,ADVANCED PEACE OFFICER,3,9.42,0.986725
77093,ROBERT,J,PERRAULT,NaN,ROBERT J PERRAULT,TEXAS DEPT OF PUBLIC SAFETY,STATE,NONE,0,0.33,0.997060
23418,JEFFERY,D,RINGSTAFF,NaN,JEFFERY D RINGSTAFF,LIBERTY HILL POLICE DEPT,WILLIAMSON,ADVANCED PEACE OFFICER,3,11.42,0.992662


## 3. Write

In [20]:
with dw.open_remote_file(DTW_PROJECT_KEY, CLEANED_FILENAME) as w:
    print("Writing to data.world:", CLEANED_FILENAME)
    officer_info.to_csv(w, index=False)

Writing to data.world: list_of_texas_officers.csv
